Based on the tutorial of this link:
http://yerevann.github.io/2017/08/25/challenges-of-reproducing-r-net-neural-network-using-keras/

# Preprocessing

In [13]:
import numpy as np
import json
import os
import argparse
import cPickle as pickle
from __future__ import print_function

In [14]:
from os import path
from gensim.scripts.glove2word2vec import glove2word2vec
from tqdm import tqdm
from unidecode import unidecode

from gensim.models import KeyedVectors
from keras.preprocessing.sequence import pad_sequences

In [15]:
from stanford_corenlp_pywrapper import CoreNLP

# utils

In [16]:
from keras.utils.data_utils import get_file

In [17]:
def custom_objects():
    from layers import *
    from model import *
    return locals()

def CoreNLP_path():
    SERVER = 'http://nlp.stanford.edu/software/'
    VERSION = 'stanford-corenlp-full-2017-06-09'
    
    origin = '{server}{version}.zip'.format(server= SERVER, version= VERSION)
    lib_dir = path.join(path.abspath(path.dirname(__file__)), 'lib')
    
    get_file('/tmp/stanford-corenlp.zip',
             origin=origin,
             cache_dir=lib_dir,
             cache_subdir='',
             extract=True)

    return path.join(lib_dir, VERSION)

def get_glove_file_path():
    SERVER = 'http://nlp.stanford.edu/data/'
    VERSION = 'glove.840B.300d'

    origin = '{server}{version}.zip'.format(server=SERVER, version=VERSION)
    cache_dir = path.join(path.abspath(path.dirname(__file__)), 'data')

    fname = '/tmp/glove.zip'
    get_file(fname,
             origin=origin,
             cache_dir=cache_dir,
             cache_subdir='',
             extract=True)

    # Remove unnecessary .zip file and keep only extracted .txt version
    os.remove(fname)
    return path.join(cache_dir, VERSION) + '.txt'

<ipython-input-17-0ff7e69f96c4>:1: SyntaxWarning: import * only allowed at module level
  def custom_objects():
<ipython-input-17-0ff7e69f96c4>:1: SyntaxWarning: import * only allowed at module level
  def custom_objects():


# Preprocessing

In [19]:
def CoreNLP_tokenizer():
    proc = CoreNLP(configdict={'annotators': 'tokenize,ssplit'},
                  corenlp_jars= [path.join(CoreNLP_path(), '*')])
    
    def tokenize_context(context):
        parsed = proc.parse_doc(context)
        tokens = []
        char_offsets = []
        for sentence in parsed['sentence']:
            tokens += sentence['tokens']
            char_offsets += sentence['char_offsets']
        return tokens, char_offsets
    return tokenize_context

def word2vec(word2vec_path):
    # Download word2vec data if it's not present yet
    if not path.exists(word2vec_path):
        glove_file_path = get_glove_file_path()
        print('Converting Glove to word2vec...', end='')
        glove2word2vec(glove_file_path, word2vec_path) # convert glove to word2vec
        os.remove(glove_file_path) # Remove glove file and keep only word2vec
        print('Done')
    print('Reading word2vec data...', end='')
    model = KeyedVectors.load_word2vec_format(word2vec_path)
    print('Done')
    
    def get_word_vector(word):
        try: 
            return model[word]
        except KeyError:
            return np.zeros(model.vector_size)
    
    return get_word_vector
    

SyntaxError: invalid syntax (<ipython-input-19-ef1a5343006a>, line 22)